# Libraries

In [13]:
from scopusScrap.scopusLogin import login
from scopusScrap.scopusUtils import log, time_formatter


import os
import time
import json
import warnings
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from habanero import Crossref

import ctypes
from tqdm.notebook import tqdm_notebook
# from ipywidgets import FloatSlider
# from IPython.display import display

from typing import Literal, Optional
# from pybliometrics.scopus.utils import config, constants
# from pybliometrics.scopus import ScopusSearch, AuthorSearch
# from pybliometrics.scopus.exception import ScopusQueryError

from fake_useragent import UserAgent
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC


# API keys

In [3]:
# API_KEY = '7f59af901d2d86f78a1fd60c1bf9426a' # ???

# API_KEY = '2486a63b5d616aa4de0576feb95e99a5'
API_KEY = '0a8e79cfa41aeded29d60f49ab30d67c'

# API_KEY = '5227c64c7de74597d4c2c75d6eb5bc16'


# APIs

In [4]:
APIS = {'abstract_citation_count' : 'https://api.elsevier.com/content/abstract/citation-count',

        'citation_overview' : 'https://api.elsevier.com/content/abstract/citations',

        'serial_title' : 'https://api.elsevier.com/content/serial/title',
        'serial_title_issn' : 'https://api.elsevier.com/content/serial/title/issn/{}',

        'subject_classifications_scidir' : 'https://api.elsevier.com/content/subject/scidir',
        'subject_classifications_scopus' : 'https://api.elsevier.com/content/subject/scopus',

        'abstract_retrieval_scopus_id' : 'https://api.elsevier.com/content/abstract/scopus_id/{}',
        'abstract_retrieval_eid' : 'https://api.elsevier.com/content/abstract/eid/{}',
        'abstract_retrieval_doi' : 'https://api.elsevier.com/content/abstract/doi/{}',
        'abstract_retrieval_pii' : 'https://api.elsevier.com/content/abstract/pii/{}',
        'abstract_retrieval_pubmed_id' : 'https://api.elsevier.com/content/abstract/pubmed_id/{}',

        'abstract_retrieval_pui' : 'https://api.elsevier.com/content/abstract/pui/{}',
        'affiliation_retrieval_affiliation_id' : 'https://api.elsevier.com/content/affiliation/affiliation_id/{}',
        'affiliation_retrieval_eid' : 'https://api.elsevier.com/content/affiliation/eid/{}',

        'author_retrieval' : 'https://api.elsevier.com/content/author',
        'author_retrieval_author_id' : 'https://api.elsevier.com/content/author/author_id/{}',
        'author_retrieval_eid' : 'https://api.elsevier.com/content/author/eid/{}',
        'author_retrieval_orcid' : 'https://api.elsevier.com/content/author/orcid/{}',

        'affiliation_search' : 'https://api.elsevier.com/content/search/affiliation',

        'author_search' : 'https://api.elsevier.com/content/search/author',

        'scopus_search' : 'https://api.elsevier.com/content/search/scopus',

        'plumx_metrics' : 'https://api.elsevier.com/analytics/plumx/{}'
        # 'plumx_metrics' : 'https://api.elsevier.com/analytics/plumx/{idType}/{idValue}'
        }


# Search

<h1> colab

In [10]:
dynamic_timeout = True
timeout_time = 1.5*60
search_within = 'ALL'
export_interval = 500
number_of_all_results = 0
number_of_allowed_retries = 3
url = 'https://www.scopus.com/search/form.uri'
*_, user_data_dir, user_agent = login()

try:
  os.chdir('Download')
except:pass
querys_txt = open('Querys.txt', 'r+')
querys = querys_txt.read().strip().split('\n')
querysname_of_file = [q for q in querys if q not in os.listdir()]
querys_txt.close()
# os.chdir('..')

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-extensions");
chrome_options.add_argument(f"user-data-dir={user_data_dir}")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--headless=new")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': r'/content/drive/MyDrive/Colab Notebooks/scopus/Download',
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

try:
    log('Search webdriver start')
    print()
    search_driver = webdriver.Chrome(options=chrome_options)
    search_driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    search_driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": user_agent})
    search_driver.get(url)

    for input_query in querys:
        flag = True
        query_start_time = time.time()
        directory = input_query
        # directory = r'D:\Coding\Uni\API\Download\{}'.format(input_query)

        log(f'Serching for "{input_query}"')
        # print(search_driver.title)
        # search_driver.refresh()

        search_within_selector = search_driver.find_element(By.CSS_SELECTOR, 'select[data-testid="select-field-select"]')
        search_within_selector.send_keys(search_within)

        query_input = search_driver.find_element(By.CSS_SELECTOR, 'input[data-testid="search-field-input"]')
        query_input.clear()
        query_input.send_keys(input_query)

        search_button = search_driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        search_button.click()

        row_element = WebDriverWait(search_driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR,
            'div[class="SearchResultsHeader-module__Qq2ZF"]')),
            'The number of results was not found!')
        result_number_element = row_element.find_element(By.CSS_SELECTOR, 'h2')
        number_of_results = int(result_number_element.text.split()[0].replace(',',''))
        log(f'{number_of_results:,} results were found')
        number_of_results = min(number_of_results, 20_000)

        list_of_range = list(range(0, number_of_results, export_interval))
        list_of_range.append(number_of_results)
        interval_list = list_of_range[1:]
        interval_list.reverse()
        first = 1
        second = interval_list.pop()

        retries = 0

        while flag:
        # while flag and retries <= number_of_allowed_retries:
            time_spent = 0
            start_time = time.time()

            # search_driver.implicitly_wait(0.5)
            while True:
                try:
                    export_button = WebDriverWait(search_driver, 7
                                      ).until(EC.presence_of_element_located(
                                          (By.CSS_SELECTOR,
                                            'div[class="export-dropdown"]')),
                                          'Dropdown menu was not found!')
                    break
                except:
                    search_driver.refresh()

            # export_button = search_driver.find_element(By.CSS_SELECTOR, 'div[class="export-dropdown"]')
            export_button.click()

            export_div = search_driver.find_element(By.CSS_SELECTOR, 'div[role="group"]')
            export_button = export_div.find_element(By.CSS_SELECTOR, 'button')
            export_button.click()

            all_result_radio_button = search_driver.find_elements(By.CSS_SELECTOR, 'input[id="select-current-page"]')[1]
            all_result_radio_button.click()

            results_start = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="From"')
            results_start.clear()
            results_start.send_keys(first)

            results_end = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="To"')
            results_end.clear()
            results_end.send_keys(second)

            checkboxes = search_driver.find_elements(By.CSS_SELECTOR, 'span[class="Checkbox-module__SaBg7"')[2:-1]
            for check in checkboxes:
                check.click()

            all_result_radio_button = search_driver.find_element(By.CSS_SELECTOR, 'button[data-testid="submit-export-button"')
            all_result_radio_button.click()

            log(f'Exporting {first:,}-{second:,} "{input_query}"')

            last_export_percent = 0
            export_progress = tqdm_notebook(total=100)

            while True:
                time.sleep(1)
                if os.path.exists('scopus.csv'):
                    try:
                        os.mkdir(directory)
                        timeout_time = int((time.time() - start_time) * 2) if dynamic_timeout else timeout_time
                    except:pass
                    name_of_file = f'{input_query}/{input_query}_{first}-{second}.csv'
                    os.rename('scopus.csv', name_of_file)
                    try:
                        number_of_results = pd.read_csv(name_of_file, low_memory=False).shape[0]
                        log(f'Download complete ({number_of_results:,} results)')
                    except:
                        number_of_results = 1j
                        log('Unable to read csv for number of results')
                    number_of_all_results += number_of_results
                    export_progress.update(100-export_progress.n)
                    export_progress.close()
                    first = second+1
                    retries = 0
                    try:
                        second = interval_list.pop()
                    except:
                        flag = False
                        log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent and {number_of_all_results} resuls were exported')
                    print()
                    break

                try:
                    time_spent = time.time() - start_time
                    if time_spent > timeout_time or search_driver.find_element(By.CSS_SELECTOR,
                            'div[role="alert"]').text.strip().lower().split()[0] == 'unable':
                        export_progress.close()

                        if time_spent > timeout_time:
                            log(f'Timeout ({timeout_time//60} minutes) reached')
                        else:
                            log('Error occured while exporting')

                        retries += 1

                        if retries < number_of_allowed_retries:
                            log(f'Retrying... (retry number {retries})')
                        elif not interval_list:
                            log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent')
                            log('Moving on to the next query')
                            flag = False
                        else:
                            first = second+1
                            second = interval_list.pop()
                            retries = 0
                            log('Skipping on to the next segment')

                        break

                    export_percentage = int(search_driver.find_element(By.CSS_SELECTOR,
                                                                        'progress').text.strip()[:-1])
                    if export_percentage != last_export_percent:
                        export_progress.update(export_percentage-last_export_percent)
                        last_export_percent = export_percentage
                except:pass
except Exception  as e:
    warnings.warn(str(e))

finally:
    search_driver.quit()

ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)

Login webdriver start                              	2023-10-15 23:31:01
You are already signed in!

Search webdriver start                             	2023-10-15 23:31:20

Serching for "natural language processing"         	2023-10-15 23:31:29
472,382 results were found                         	2023-10-15 23:31:37
Exporting 1-500 "natural language processing"      	2023-10-15 23:31:38


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
multiplier = 3
dynamic_timeout = True
timeout_time = 5*60
search_within = 'ALL'
export_interval = 500
number_of_all_results = 0
number_of_allowed_retries = 3
url = 'https://www.scopus.com/search/form.uri'
*_, user_data_dir, user_agent = login()

if not os.getcwd().endswith('\\Download'):
  os.chdir('Download')

querys_txt = open('Querys.txt', 'r+')
querys = querys_txt.read().strip().split('\n')
querys = [q for q in querys if q not in os.listdir()]
querys_txt.close()

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-extensions");
chrome_options.add_argument(f"user-data-dir={user_data_dir}")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--headless=new")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': os.getcwd(),
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

# ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

try:
    log('Search webdriver start')
    print()
    search_driver = webdriver.Chrome(options=chrome_options)
    search_driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    search_driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": user_agent})
    search_driver.get(url)

    for input_query in querys:
        flag = True
        query_start_time = time.time()
        directory = input_query
        # directory = r'D:\Coding\Uni\API\Download\{}'.format(input_query)

        log(f'Serching for "{input_query}"')
        # print(search_driver.title)
        # search_driver.refresh()

        search_within_selector = search_driver.find_element(By.CSS_SELECTOR, 'select[data-testid="select-field-select"]')
        search_within_selector.send_keys(search_within)

        query_input = search_driver.find_element(By.CSS_SELECTOR, 'input[data-testid="search-field-input"]')
        query_input.clear()
        query_input.send_keys(input_query)

        search_button = search_driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        search_button.click()

        row_element = WebDriverWait(search_driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR,
            'div[class="SearchResultsHeader-module__Qq2ZF"]')),
            'The number of results was not found!')
        result_number_element = row_element.find_element(By.CSS_SELECTOR, 'h2')
        number_of_results = int(result_number_element.text.split()[0].replace(',',''))
        log(f'{number_of_results:,} results were found')
        number_of_results = min(number_of_results, 20_000)

        list_of_range = list(range(0, number_of_results, export_interval))
        list_of_range.append(number_of_results)
        interval_list = list_of_range[1:]
        interval_list.reverse()
        first = 1
        second = interval_list.pop()

        retries = 0

        while flag:
        # while flag and retries <= number_of_allowed_retries:
            time_spent = 0
            start_time = time.time()

            for _ in range(number_of_allowed_retries):
                try:
                    export_button = WebDriverWait(search_driver, 7
                                      ).until(EC.presence_of_element_located(
                                          (By.CSS_SELECTOR,
                                            'div[class="export-dropdown"]')),
                                          'Dropdown menu was not found!')
                    break
                except:
                    log('Dropdown menu was not found')
                    search_driver.refresh()

            # export_button = search_driver.find_element(By.CSS_SELECTOR, 'div[class="export-dropdown"]')
            export_button.click()

            export_div = search_driver.find_element(By.CSS_SELECTOR, 'div[role="group"]')
            export_button = export_div.find_element(By.CSS_SELECTOR, 'button')
            export_button.click()

            all_result_radio_button = search_driver.find_elements(By.CSS_SELECTOR, 'input[id="select-current-page"]')[1]
            all_result_radio_button.click()

            results_start = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="From"')
            results_start.clear()
            results_start.send_keys(first)

            results_end = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="To"')
            results_end.clear()
            results_end.send_keys(second)

            checkboxes = search_driver.find_elements(By.CSS_SELECTOR, 'span[class="Checkbox-module__SaBg7"')[2:-1]
            for check in checkboxes:
                check.click()

            all_result_radio_button = search_driver.find_element(By.CSS_SELECTOR, 'button[data-testid="submit-export-button"')
            all_result_radio_button.click()

            log(f'Exporting {first:,}-{second:,} "{input_query}"')

            last_export_percent = 0
            export_progress = tqdm_notebook(total=100)

            while True:
                time.sleep(1)
                if os.path.exists('scopus.csv'):
                    number_of_results = pd.read_csv('scopus.csv', low_memory=False).shape[0]
                    try:
                        os.mkdir(directory)
                        timeout_time = int((time.time() - start_time) * multiplier) if dynamic_timeout else timeout_time
                        log(f'Timeout is {timeout_time//60} minutes')
                    except:pass
                    os.rename('scopus.csv', f'{input_query}\\{input_query}_{first}-{second}.csv')
                    log(f'Download complete ({number_of_results:,} results)')
                    number_of_all_results += number_of_results
                    export_progress.update(100-export_progress.n)
                    export_progress.close()
                    first = second+1
                    retries = 0
                    try:
                        second = interval_list.pop()
                    except:
                        flag = False
                        log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent')
                    print()
                    break

                try:
                    time_spent = time.time() - start_time
                    if time_spent > timeout_time or search_driver.find_element(By.CSS_SELECTOR,
                            'div[role="alert"]').text.strip().lower().split()[0] == 'unable':
                        export_progress.close()

                        if time_spent > timeout_time:
                            log(f'Timeout ({timeout_time//60} minutes) reached')
                        else:
                            log('Error occured while exporting')

                        retries += 1

                        if retries < number_of_allowed_retries:
                            log(f'Retrying... (retry number {retries})')
                        elif not interval_list:
                            log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent')
                            log('Moving on to the next query')
                            flag = False
                        else:
                            first = second+1
                            second = interval_list.pop()
                            retries = 0
                            log('Skipping on to the next segment')

                        break

                    export_percentage = int(search_driver.find_element(By.CSS_SELECTOR,
                                                                        'progress').text.strip()[:-1])
                    if export_percentage != last_export_percent:
                        export_progress.update(export_percentage-last_export_percent)
                        last_export_percent = export_percentage
                except:pass
except Exception  as e:
    warnings.warn(str(e))

search_driver.quit()

ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)


KeyboardInterrupt: 

In [55]:
timeout_time = 5*60
export_interval = 1_000
number_of_all_results = 0
number_of_allowed_retries = 3

user_agent = login(user_data_dir=r'D:\Coding\Uni\API\selenium')[-1]
os.chdir(r'D:\Coding\Uni\API\Download')

querys_txt = open('Querys.txt', 'r+')
querys = querys_txt.read().strip().split('\n')
querys = [q for q in querys if q not in os.listdir()]
querys_txt.close()

url = 'https://www.scopus.com/search/form.uri'
search_within = 'ALL'

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-extensions");
chrome_options.add_argument(r"user-data-dir=D:\Coding\Uni\API\selenium")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--headless=new")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': r'D:\Coding\Uni\API\Download',
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

try:
    log('Search webdriver start')
    print()
    search_driver = webdriver.Chrome(options=chrome_options)
    search_driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    search_driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": user_agent}) 
    search_driver.get(url)

    for input_query in querys:
        flag = True
        query_start_time = time.time()
        directory = r'D:\Coding\Uni\API\Download\{}'.format(input_query)

        log(f'Serching for "{input_query}"')
        # print(search_driver.title)
#         search_driver.refresh()

        search_within_selector = search_driver.find_element(By.CSS_SELECTOR, 'select[data-testid="select-field-select"]')
        search_within_selector.send_keys(search_within)

        query_input = search_driver.find_element(By.CSS_SELECTOR, 'input[data-testid="search-field-input"]')
        query_input.clear()
        query_input.send_keys(input_query)

        search_button = search_driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        search_button.click()

        row_element = WebDriverWait(search_driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR,
            'div[class="SearchResultsHeader-module__Qq2ZF"]')),
            'The number of results was not found!')
        result_number_element = row_element.find_element(By.CSS_SELECTOR, 'h2')
        number_of_results = int(result_number_element.text.split()[0].replace(',',''))
        log(f'{number_of_results:,} results were found')
        number_of_results = min(number_of_results, 20_000)

        list_of_range = list(range(0, number_of_results, export_interval))
        list_of_range.append(number_of_results)
        interval_list = list_of_range[1:]
        interval_list.reverse()
        first = 1
        second = interval_list.pop()

        retries = 0

        while flag:
#         while flag and retries <= number_of_allowed_retries:
            time_spent = 0
            start_time = time.time()

#             search_driver.implicitly_wait(0.5)
            while True:
                try:
                    export_button = WebDriverWait(search_driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR,
                                            'div[class="export-dropdown"]')),
                                            'Dropdown menu was not found!')
                    break
                except:
                    search_driver.refresh()

#             export_button = search_driver.find_element(By.CSS_SELECTOR, 'div[class="export-dropdown"]')
            export_button.click()

            export_div = search_driver.find_element(By.CSS_SELECTOR, 'div[role="group"]')
            export_button = export_div.find_element(By.CSS_SELECTOR, 'button')
            export_button.click()

            all_result_radio_button = search_driver.find_elements(By.CSS_SELECTOR, 'input[id="select-current-page"]')[1]
            all_result_radio_button.click()

            results_start = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="From"')
            results_start.clear()
            results_start.send_keys(first)

            results_end = search_driver.find_element(By.CSS_SELECTOR, 'input[placeholder="To"')
            results_end.clear()
            results_end.send_keys(second)

            checkboxes = search_driver.find_elements(By.CSS_SELECTOR, 'span[class="Checkbox-module__SaBg7"')[2:-1]
            for check in checkboxes:
                check.click()

            all_result_radio_button = search_driver.find_element(By.CSS_SELECTOR, 'button[data-testid="submit-export-button"')
            all_result_radio_button.click()

            log(f'Exporting {first:,}-{second:,} "{input_query}"')

            last_export_percent = 0
            export_progress = tqdm_notebook(total=100)

            while True:
                time.sleep(1)
                if os.path.exists('scopus.csv'):
                    number_of_results = pd.read_csv('scopus.csv', low_memory=False).shape[0]
                    try:
                        os.mkdir(directory)
                    except:pass
                    os.rename('scopus.csv', f'{input_query}\\{input_query}_{first}-{second}.csv')
                    log(f'Download complete ({number_of_results:,} results)')
                    number_of_all_results += number_of_results
                    export_progress.update(100-export_progress.n)
                    export_progress.close()
                    first = second+1
                    retries = 0
                    try:
                        second = interval_list.pop()
                    except:
                        flag = False
                        log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent')
                    print()
                    break

                try:
                    time_spent = time.time() - start_time
                    if time_spent > timeout_time or search_driver.find_element(By.CSS_SELECTOR,
                            'div[role="alert"]').text.strip().lower().split()[0] == 'unable':
                        export_progress.close()

                        if time_spent > timeout_time:
                            log(f'Timeout ({timeout_time//60} minutes) reached')
                        else:
                            log('Error occured while exporting')

                        retries += 1

                        if retries < number_of_allowed_retries:
                            log(f'Retrying... (retry number {retries})')
                        elif not interval_list:
                            log(f'Exporting "{input_query}" has ended. {time_formatter(query_start_time)} was spent')
                            log('Moving on to the next query')
                            flag = False
                        else:
                            first = second+1
                            second = interval_list.pop()
                            retries = 0                   
                            log('Skipping on to the next segment')

                        break

                    export_percentage = int(search_driver.find_element(By.CSS_SELECTOR,
                                                                        'progress').text.strip()[:-1])
                    if export_percentage != last_export_percent:
                        export_progress.update(export_percentage-last_export_percent)
                        last_export_percent = export_percentage
                except:pass
except Exception  as e:
    warnings.warn(str(e))

search_driver.quit()
ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)


Login webdriver start                              	2023-10-08 11:29:47
Password entered                                   	2023-10-08 11:29:55
Scopus - Document search | Signed in

Search webdriver start                             	2023-10-08 11:30:05

Serching for "machine learning"                    	2023-10-08 11:30:13
2,518,854 results were found                       	2023-10-08 11:30:17
Exporting 1-1000 "machine learning"                	2023-10-08 11:30:22


  0%|          | 0/100 [00:00<?, ?it/s]

Download complete (1,000 results)                  	2023-10-08 11:31:39

Exporting 1001-2000 "machine learning"             	2023-10-08 11:31:40


  0%|          | 0/100 [00:00<?, ?it/s]

Download complete (1,000 results)                  	2023-10-08 11:32:54

Exporting 2001-3000 "machine learning"             	2023-10-08 11:32:55


...

Download complete (1,000 results)                  	2023-10-08 12:02:59

Exporting 19001-20000 "machine learning"           	2023-10-08 12:03:00


  0%|          | 0/100 [00:00<?, ?it/s]

Download complete (1,000 results)                  	2023-10-08 12:04:43
Exporting "machine learning" has ended. 00h 34m 30s was spent 	2023-10-08 12:04:43

Serching for ""                                    	2023-10-08 12:04:43
2,518,854 results were found                       	2023-10-08 12:04:44
Exporting 1-1000 ""                                	2023-10-08 12:04:45


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [56]:
search_driver.quit()


In [57]:
querys

['machine learning', '']

# Cited by

In [ ]:
os.chdir(r'D:\Coding\Uni\API\Download')
searched_querys = [dr for dr in os.listdir() if os.path.isdir(dr)]

try:
    chrome_options = Options()
    chrome_options.add_argument(r"user-data-dir=D:\Coding\Uni\API\selenium")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_experimental_option('prefs', {
        'download.default_directory': r'D:\Coding\Uni\API\Download',
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
    })

    log('Webdriver start')
    print()
    cited_by_driver = webdriver.Chrome(options=chrome_options)

    for citing_search in searched_querys:
        query_df = pd.read_csv(f"{citing_search}\\{citing_search}.csv")
        query_df = query_df[query_df.loc[:, 'Cited by'] != 0].loc[:, ['DOI', 'Link']]

        for doi, url in query_df.values:
            log(f"Searching for DOI: {doi} cited by's")
            cited_by_driver.get(url)

            cited_bys_link = cited_by_driver.find_element(By.CSS_SELECTOR, 'a[title="View all citing documents"]')
            cited_bys_link.click()

            cited_by_driver.implicitly_wait(0.5)

            select_all_results = cited_by_driver.find_element(By.CSS_SELECTOR, 'li[id="selectAllCheck"]')
            select_all_results.click()

            export_buttons = cited_by_driver.find_element(By.CSS_SELECTOR,
                    'span[id="chunkExport"]').find_elements(By.CSS_SELECTOR, 'button')
            export_incomplete_results, export_options = export_buttons

            export_options.click()

            select_all = cited_by_driver.find_element(By.CSS_SELECTOR, 'th[id="otherInformationCheckboxes"]')
            select_all.click()

            export_all_results = cited_by_driver.find_element(By.CSS_SELECTOR, 'button[id="exportTrigger"]')
            export_all_results.click()

            log(f'Exporting citation results')

            while True:
                if os.path.exists('scopus.csv'):
                    sanitized_doi = doi.replace('/', '_')
                    os.mkdir(f'{citing_search}\\{sanitized_doi}')
                    os.rename('scopus.csv', f'{citing_search}\\{sanitized_doi}\\{sanitized_doi}.csv')
                    log('Download complete')
                    print()
                    break

except Exception  as e:
    warnings.warn(str(e))

cited_by_driver.quit()


# References one by one

In [ ]:
from habanero import Crossref
cr = Crossref()

os.chdir(r'D:\Coding\Uni\API\Download')
searched_querys = [dr for dr in os.listdir() if os.path.isdir(dr)][:1]

try:
    for reference_folder in searched_querys:
        log(f'Searching "{reference_folder}" folder')

        for reference_csv in os.listdir(reference_folder)[:2]:
            query_df = pd.read_csv(f"{reference_folder}\\{reference_csv}").dropna(subset='References')
            query_df_values = query_df.loc[:, ['References']].values
            column_values = []

            for references in tqdm_notebook(query_df_values):
                row_reference = []

                for reference in tqdm_notebook(references[0].split(';')):
                    row_reference.append(cr.works(query = reference, limit=1)['message']['items'][0]['DOI'])
                column_values.append('; '.join(row_reference))

            query_df['References DOI'] = column_values
            query_df.to_csv(f"{reference_folder}\\DOI_{reference_csv}")
            del query_df
            log(f'Closed "{reference_csv}"')

except Exception  as e:
    warnings.warn(str(e))


# References all at once

In [16]:
cr = Crossref()
os.chdir(r'D:\Coding\Uni\API\Download')
searched_querys = [dr for dr in os.listdir() if os.path.isdir(dr)][:1]

for csv_folder in searched_querys:
    log(f'Searching "{csv_folder}" folder')
    csv_files = os.listdir(csv_folder)
    csv_files = [file for file in csv_files if f'DOI_search_{file}' not in csv_files and not file.startswith('DOI_search_')][:]

    for csv_file in csv_files:
        query_df = pd.read_csv(f"{csv_folder}\\{csv_file}").dropna(subset='DOI')
        query_df_dois = [doi_str for doi_str in query_df.loc[:, ['DOI']].astype(str).values.reshape(-1).tolist()]
        
        log(f'Processing "{csv_file}"')

#         overall_unfound = 0
        references_column_values = []
        actual_number_of_references_column = []
        number_of_found_references_column = []
        number_of_unfound_references_column = []

        for doi in tqdm_notebook(query_df_dois):
            try:
                article_references = cr.works(ids=doi)['message']['reference']
            except Exception as e:
#                 log(str(e))
#                 log(f'Cant fetch data for DOI: "{doi}"')
                references_column_values.append(None)
                actual_number_of_references_column.append(None)
                number_of_found_references_column.append(None)
                number_of_unfound_references_column.append(None)
            else:
                # doi_column_values.append(doi)
                result_LIST = [reference_doi['DOI'] for reference_doi in article_references if reference_doi.get('DOI')]

                references_column_values.append('; '.join(result_LIST))
                actual_number_of_references_column.append(len(article_references))
                number_of_found_references_column.append(len(result_LIST))
                number_of_unfound_references_column.append(len(article_references) - len(result_LIST))

        query_df['References DOI all'] = references_column_values
        query_df['Actual number of references'] = actual_number_of_references_column
        query_df['Found references'] = number_of_found_references_column
        query_df['Unfound references'] = number_of_unfound_references_column

        query_df.to_csv(f"{csv_folder}\\DOI_search_{csv_file}")
        del query_df
        log(f'Processing "{csv_file}" has finished')
        log(f'Article note found: {number_of_unfound_references_column.count(None)}')
        log(f'Unfound references: {sum([number for number in number_of_unfound_references_column if number])}')


Searching "attention" folder                	2023-10-17 17:33:02
Processing "attention_10001-11000.csv"      	2023-10-17 17:33:02


  0%|          | 0/1000 [00:00<?, ?it/s]

Processing "attention_10001-11000.csv" has finished 	2023-10-17 17:47:27
Processing "attention_1001-2000.csv"        	2023-10-17 17:47:27


  0%|          | 0/999 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Query formatter

In [11]:
def query_formatter(input_str: str) -> str:
    """
    Format a search query string for use in Scopus API requests.

    This function takes an input search query string and performs formatting to make it
    suitable for use in Scopus API requests. It converts the input string to lowercase,
    replaces specific characters, and ensures proper use of boolean operators.

    Args:
        input_str (str): The input search query string to be formatted.

    Returns:
        str: The formatted search query string.

    Example:
        >>> formatted_query = query_formatter('machine learning AND neural networks')
        >>> print(formatted_query)
        'machine+learning+neural+networks'

    Notes:
        - The function converts the input query to lowercase to ensure case-insensitive
          searching.
        - It replaces 'and', 'or', 'not', '+', '-', and '~' with the appropriate boolean
          operators ('AND', 'OR', 'NOT') for Scopus API queries.
        - Multiple spaces are replaced with a single space, and the query is joined
          using '+' to represent spaces in the final formatted query.

    See Also:
        - For Scopus API usage, you can use the formatted query as a parameter in API
          requests to retrieve specific data.
    """
    output_str = input_str.lower()

    replace_chars = {
        ' and ': ' ',
        ' or ': ' ',
        ' not ': ' ',
        ' + ': ' AND ',
        ' - ': ' NOT ',
        ' ~ ': ' OR ',
    }

    for key, value in replace_chars.items():
        output_str = output_str.replace(key, value)

    return "+".join(output_str.split())
    # return f'key({"+".join(output_str.split())})'


# API function

In [9]:
def scopus_apis(
    query: Optional[str] = None,
    api_link: Literal[
        'abstract_citation_count', 'citation_overview', 'serial_title',
        'serial_title_issn', 'subject_classifications_scidir',
        'subject_classifications_scopus', 'abstract_retrieval_scopus_id',
        'abstract_retrieval_eid', 'abstract_retrieval_doi',
        'abstract_retrieval_pii', 'abstract_retrieval_pubmed_id',
        'abstract_retrieval_pui', 'affiliation_retrieval_affiliation_id',
        'affiliation_retrieval_eid', 'author_retrieval',
        'author_retrieval_author_id', 'author_retrieval_eid',
        'author_retrieval_orcid', 'affiliation_search',
        'author_search', 'scopus_search', 'plumx_metrics'
    ] = 'scopus_search',
    requirement: Optional[str] = None,
    parameters: dict = {},
    show_result: bool = True,
    reformat_query: bool = False,
    accept: Literal['image/jpeg', 'application/json', 'text/xml', 'application/xml', 'text/html'] = 'application/json'
) -> requests.models.Response:
    """
    Retrieve data from various APIs based on the provided API name or URL.

    Args:
        query (str, None): Additional query parameters for the API request.

        api_link (str): The API name or URL to fetch data from. It should be a string
            representing either a valid API name from the list of available APIs or
            a complete API URL. Default is 'scopus_search'.

        requirement (str, None): Additional requirement for the API request.

        parameters (dict): Filter parameters.

        show_result (bool): Whether to print the API response. Default is True.

        reformat_query (bool): Whether to reform query into the right format.

    Returns:
        requests.Response: The response object from the API request.

    Examples:
        To retrieve data from the Scopus search API, use:
        >>> scopus_apis('scopus_search')

        To retrieve data from a custom API with a specific URL, use:
        >>> scopus_apis('https://customapi.example.com/data')

    Raises:
        ValueError: If the provided `api_link` is not a string or is not a valid
            API name. Also, raises a ValueError if the API requires an argument
            (`requirement`) but it's not provided.

    Notes:
        - The function supports multiple APIs, and you can extend the list of APIs and
          their corresponding URLs in the `APIS` dictionary.
        - Ensure that the provided API name is one of the keys in the `APIS` dictionary
          to fetch data from the correct source.
    """
    global session

    if query and reformat_query:
        query = query_formatter(query)

    if APIS[api_link][-2:] == '{}' and not requirement:
        raise ValueError('This API needs an argument (requirement).')

    api_link_str = APIS[api_link].format(requirement) if requirement else APIS[api_link]
    query_str = f'?query={query}' if query else ''

    resp = requests.get(api_link_str + query_str,
                    headers={'Accept': accept,
                             'X-ELS-APIKey': API_KEY},
                    params=parameters)

    if show_result:
        decorative_lines = f'{"="*(len(api_link)+2)}\n'
        print(f'{decorative_lines} {api_link.upper()} \n{decorative_lines}')

        if accept == 'application/json':
            print(json.dumps(resp.json(),
                            sort_keys=True,
                        indent=4, separators=(',', ': ')))
        else:
            print(resp)

    return resp


# Authenticate

In [8]:
resp = session.get("https://api.elsevier.com/authenticate/",
                headers={'Accept':'application/json',
                            'X-ELS-APIKey': API_KEY})

print(json.dumps(resp.json(),
                sort_keys=True,
                indent=4, separators=(',', ': ')))


{
    "service-error": {
        "status": {
            "statusCode": "AUTHENTICATION_ERROR",
            "statusText": "Requestor configuration settings insufficient for access to this resource."
        }
    }
}


# Testing all APIs

In [11]:
print(scopus_apis('10.1016/S0014-5793(01)03313-0', 'abstract_citation_count', show_result=False))                                         # AUTHENTICATION_ERROR
print(scopus_apis('10.1016/S0014-5793(01)03313-0', 'citation_overview', show_result=False))                                               # AUTHENTICATION_ERROR
print(scopus_apis('gene', 'serial_title', show_result=False))
print(scopus_apis(requirement='03781119', api_link='serial_title_issn', show_result=False))
print(scopus_apis(api_link= 'subject_classifications_scidir', show_result=False))
print(scopus_apis(api_link= 'subject_classifications_scopus', show_result=False))
print(scopus_apis(requirement='0037070197',  api_link='abstract_retrieval_scopus_id', show_result=False))
print(scopus_apis(requirement='2-s2.0-0037070197', api_link='abstract_retrieval_eid', show_result=False))
print(scopus_apis(requirement='10.1016/S0014-5793(01)03313-0', api_link='abstract_retrieval_doi', show_result=False))
print(scopus_apis(requirement='S0014579301033130', api_link='abstract_retrieval_pii', show_result=False))
print(scopus_apis(requirement='11852050', api_link='abstract_retrieval_pubmed_id', show_result=False))
print(scopus_apis(requirement='34164449', api_link='abstract_retrieval_pui', show_result=False))
print(scopus_apis(requirement='60016849', api_link='affiliation_retrieval_affiliation_id', show_result=False))                            # AUTHORIZATION_ERROR
print(scopus_apis(requirement='10-s2.0-60016849', api_link='affiliation_retrieval_eid', show_result=False))                               # AUTHORIZATION_ERROR
print(scopus_apis('22988279600', 'author_retrieval', show_result=False))                                                                  # AUTHORIZATION_ERROR
print(scopus_apis(requirement='22988279600', api_link='author_retrieval_author_id', show_result=False))                                   # AUTHORIZATION_ERROR
print(scopus_apis(requirement='9-s2.0-22988279600', api_link='author_retrieval_eid', show_result=False))                                  # AUTHORIZATION_ERROR
print(scopus_apis(requirement='0000-0003-4700-6699', api_link='author_retrieval_orcid', show_result=False))                               # AUTHORIZATION_ERROR
print(scopus_apis('affil(Broad Institute)', 'affiliation_search', show_result=False))                                                     # AUTHORIZATION_ERROR
print(scopus_apis('authlast(Einstein) + authfirst(Albert) + affil(Princeton)', 'author_search', reformat_query=True, show_result=False))  # AUTHORIZATION_ERROR
print(scopus_apis('all(gene)', show_result=False))
print(scopus_apis(requirement='doi/10.1016/S0014-5793(01)03313-0', api_link='plumx_metrics', show_result=False))                          # AUTHENTICATION_ERROR


<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [200]>
<Response [401]>


# First 5'000 results

In [34]:
results = []
batch_size = 25
query = 'key(NLP)'

try:
    num_results = ScopusSearch(query, subscriber=False).get_results_size()
except ScopusQueryError:
    num_results = 25
    # num_results = 5000

for start in range(0, num_results, batch_size):
    resp = scopus_apis(f"{query}&start={start}&count="+
            f"{min(batch_size, num_results-start)}&sort=citedby-count")

    results.append(resp)
    # results.append(resp.json())


 SCOPUS_SEARCH 

{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "affiliation": [
                    {
                        "@_fa": "true",
                        "affiliation-city": "Montreal",
                        "affiliation-country": "Canada",
                        "affilname": "University of Montreal"
                    }
                ],
                "citedby-count": "1418",
                "dc:creator": "Turian J.",
                "dc:identifier": "SCOPUS_ID:80053495924",
                "dc:title": "Word representations: A simple and general method for semi-supervised learning",
                "eid": "2-s2.0-80053495924",
                "link": [
                    {
                        "@_fa": "true",
                        "@href": "https://api.elsevier.com/content/abstract/scopus_id/80053495924",
                        "@ref": "self"
                    },
                    {
            

# Scrap functions

<h2> Abstract

In [53]:
def get_abstract (url: str, signed_in: bool) -> str:

    global session, headers, username, password

    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # with open('yo.html', 'w') as f:
    #     f.write(response.text)

    text_container = soup.find('section').find('p') if signed_in else soup.find('section', id="abstractSection").find('p')
    extracted_text = text_container.get_text()

    return extracted_text


<h2> Cited by

In [78]:
def get_cited_by(url: str, signed_in: bool) -> list:
    global session, headers, username, password

    response = session.get(url, headers=headers,timeout=15)
    soup = BeautifulSoup(response.text, "html.parser")

    rows = []
    rows_raw = soup.find_all("tr", class_="searchArea")

    columns = [
                    element.get_text().strip()
                    for element in soup.find('table',
                    id='srchResultsList').find('tr').find_all("th")[1:]
                ] if signed_in else [
                    element.get_text()
                    for element in soup.find('thead').find_all("th")[1:]
                ]

    for row in rows_raw:

        row_extracted = {key:value.get_text().strip()
                            for key, value in zip(columns, row.find_all('td'))}
        rows.append(row_extracted)

    return rows


# Scrap data from results

In [79]:
all_data = []

for page in results:
    page_text = json.loads(page.text.encode('utf-8'))

    for result in page_text["search-results"]["entry"]:

        if result["citedby-count"]:
            data = []

            for link in result['link']:

                # if link["@ref"]  == "scopus":
                #     temp = get_abstract(link["@href"], True)
                #     data.append(temp)

                if link["@ref"] == "scopus-citedby":
                    temp = get_cited_by(link["@href"], True)
                    data.append(temp)

            if data:
                all_data.append(data)


In [81]:
len(all_data)


25

# ***DELETE*** (tests)

In [ ]:
def get_cited_by(url: str):
    global session, headers, username, password

    response = session.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    doc_titles = []
    author_lists = []
    publication_years = []
    sources = []

    rows = soup.find_all("tr", class_="searchArea")

    for row in rows:
        doc_title = row.find("td", data_type="docTitle").find("a", class_="ddmDocTitle").text
        author_list = [a.text for a in row.find("span", class_="ddmAuthorList").find_all("a")]
        pub_year = row.find("td", class_="textRight").find("span", class_="ddmPubYr").text
        source = row.find("td", data_type="source").find("a", class_="ddmDocSource").text

        doc_titles.append(doc_title)
        author_lists.append(author_list)
        publication_years.append(pub_year)
        sources.append(source)

    for i in range(len(doc_titles)):
        print(f"Title: {doc_titles[i]}" +
              f"Authors: {', '.join(author_lists[i])}" +
              f"Publication Year: {publication_years[i]}" +
              f"Source: {sources[i]}\n")


In [ ]:
# login()

# def get_cited_by(url: str):
def get_cited_by(url: str= "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=80053495924&origin=inward"):

    # URL of the page to scrape
    # url = "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=80053495924&origin=inward"

    # Send an HTTP GET request to the URL
    response = session.get(url, headers=headers, auth=(username, password))

    # Parse the HTML content
    soup = BeautifulSoup(response.text, "html.parser")

    # Initialize lists to store scraped data
    doc_titles = []
    author_lists = []
    publication_years = []
    sources = []

    # Find all rows with class "searchArea"
    rows = soup.find_all("tr", class_="searchArea")

    # Loop through each row to extract data
    for row in rows:
        doc_title = row.find("td", data_type="docTitle").find("a", class_="ddmDocTitle").text
        author_list = [a.text for a in row.find("span", class_="ddmAuthorList").find_all("a")]
        pub_year = row.find("td", class_="textRight").find("span", class_="ddmPubYr").text
        source = row.find("td", data_type="source").find("a", class_="ddmDocSource").text

        doc_titles.append(doc_title)
        author_lists.append(author_list)
        publication_years.append(pub_year)
        sources.append(source)

    # Print the scraped data
    for i in range(len(doc_titles)):
        print(f"Title: {doc_titles[i]}")
        print(f"Authors: {', '.join(author_lists[i])}")
        print(f"Publication Year: {publication_years[i]}")
        print(f"Source: {sources[i]}")
        print("\n")

get_cited_by()


In [47]:
# response
rows[0]
# print(rows[0].find("span", class_="docTitle").text)


<tr class="searchArea" id="resultDataRow0">
<th rowspan="2" scope="row">
<div class="checkbox disabled">
<input disabled="true" id="eid_2-s2.0-85151219234" name="selectedEIDs" onclick="selectDeselectResult(this.form, this)" type="checkbox" value="2-s2.0-85151219234">
<label class="checkbox-label" for="eid_2-s2.0-85151219234">
1
</label>
</input></div>
</th>
<td>
<span class="docTitle">
A corpus of CO<inf>2</inf> electrocatalytic reduction process extracted from the scientific literature 
</span>
<br/>
<div class="openAccessTxt">
Open Access
</div>
</td>
<td>
<span class="previewTxt" title="Show author details: Subscription required">Wang, L.</span><span style="color:#C0C0C0;">, </span><span class="previewTxt" title="Show author details: Subscription required">Gao, Y.</span><span style="color:#C0C0C0;">, </span><span class="previewTxt" title="Show author details: Subscription required">Chen, X.</span><span style="color:#C0C0C0;">, </span><span style="color:#C0C0C0;">(...), </span><span 

# pybliometrics

<h2> APIs

In [ ]:
constants.CONFIG_FILE

print(config['Authentication']['APIKey'])

5227c64c7de74597d4c2c75d6eb5bc16


# Other

In [ ]:
def get_scopus_info(SCOPUS_ID):
    url = ("http://api.elsevier.com/content/abstract/scopus_id/"
          + SCOPUS_ID
          + "?field=authors,title,publicationName,volume,issueIdentifier,"
          + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
    resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': API_KEY})

    print(json.dumps(resp.json(),
                 sort_keys=True,
                 indent=4, separators=(',', ': ')))

    results = json.loads(resp.text.encode('utf-8'))

    ARR = 'abstracts-retrieval-response'

    # authors=', '.join([au['ce:indexed-name'] for au in results[ARR]['authors']['author']])
    authors= ''
    title=results[ARR]['coredata']['dc:title']
    journal=results[ARR]['coredata']['prism:publicationName']
    volume=results[ARR]['coredata']['prism:volume']
    articlenum=(results[ARR]['coredata'].get('prism:pageRange') or results[ARR]['coredata'].get('article-number'))
    date=results[ARR]['coredata']['prism:coverDate']
    doi=f"doi:{results[ARR]['coredata']['prism:doi']}"
    cites=int(results[ARR]['coredata']['citedby-count'])

    return f'{authors},\n{title},\nJournal: {journal}, Volume: {volume}, '\
f'Article number: {articlenum}, ({date}).\n{doi} (cited {cites} times).\n'

print(get_scopus_info('SCOPUS_ID:0037368024'))

In [ ]:
# def get_scopus_info(SCOPUS_ID):
#     url = ("http://api.elsevier.com/content/abstract/scopus_id/"
#           + SCOPUS_ID
#           + "?field=authors,title,publicationName,volume,issueIdentifier,"
#           + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
#     resp = requests.get(url,
#                     headers={'Accept':'application/json',
#                              'X-ELS-APIKey': API_KEY})

#     return json.loads(resp.text.encode('utf-8'))


# i = 0
# for sid in scopus_ids:
#     # some entries seem to have json parse errors, so we catch those
#     try:
#         results = get_scopus_info(sid[0])  # index 0 because the input data is a 2d array
#         if results['abstracts-retrieval-response']['coredata']['prism:aggregationType'] == 'Journal':
#             i += 1
#             fstring = '{authors}, {title}, {journal}, {volume}, {articlenum}, ({date}). <a href="https://doi.org/{doi}">{doi}</a> (cited {cites} times)\n\n'

#             s = fstring.format(authors=', '.join([au['ce:indexed-name'].encode('utf-8') for au in results['abstracts-retrieval-response']['authors']['author']]),
#                                title=results['abstracts-retrieval-response']['coredata']['dc:title'].encode('utf-8'),
#                                journal=results['abstracts-retrieval-response']['coredata']['prism:publicationName'].encode('utf-8'),
#                                volume=results['abstracts-retrieval-response']['coredata'].get('prism:volume', 'None').encode('utf-8'),
#                                articlenum=str((results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
#                                            results['abstracts-retrieval-response']['coredata'].get('article-number'))).encode('utf-8'),
#                                date=results['abstracts-retrieval-response']['coredata']['prism:coverDate'].encode('utf-8'),
#                                doi='doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi'].encode('utf-8'),
#                                cites=int(results['abstracts-retrieval-response']['coredata']['citedby-count'].encode('utf-8')))
#             print('{0:3d}. {1}<br>'.format(i, s))
#     except:
#         print('{0:3d}. {1}'.format(i, sid))